In [37]:
import polars as pl

In [38]:
df = pl.read_csv("aac_intakes_outcomes.csv")

print(df.shape)
df.head()


(79672, 41)


age_upon_outcome,animal_id_outcome,date_of_birth,outcome_subtype,outcome_type,sex_upon_outcome,age_upon_outcome_(days),age_upon_outcome_(years),age_upon_outcome_age_group,outcome_datetime,outcome_month,outcome_year,outcome_monthyear,outcome_weekday,outcome_hour,outcome_number,dob_year,dob_month,dob_monthyear,age_upon_intake,animal_id_intake,animal_type,breed,color,found_location,intake_condition,intake_type,sex_upon_intake,count,age_upon_intake_(days),age_upon_intake_(years),age_upon_intake_age_group,intake_datetime,intake_month,intake_year,intake_monthyear,intake_weekday,intake_hour,intake_number,time_in_shelter,time_in_shelter_days
str,str,str,str,str,str,i64,f64,str,str,i64,i64,str,str,i64,f64,i64,i64,str,str,str,str,str,str,str,str,str,str,i64,i64,f64,str,str,i64,i64,str,str,i64,f64,str,f64
"""10 years""","""A006100""","""2007-07-09 00:00:00""",null,"""Return to Owner""","""Neutered Male""",3650,10.0,"""(7.5, 10.0]""","""2017-12-07 14:07:00""",12,2017,"""2017-12""","""Thursday""",0,1.0,2007,7,"""2017-12""","""10 years""","""A006100""","""Dog""","""Spinone Italiano Mix""","""Yellow/White""","""Colony Creek And Hunters Trace…","""Normal""","""Stray""","""Neutered Male""",1,3650,10.0,"""(7.5, 10.0]""","""2017-12-07 00:00:00""",12,2017,"""2017-12""","""Thursday""",14,1.0,"""0 days 14:07:00.000000000""",0.588194
"""7 years""","""A006100""","""2007-07-09 00:00:00""",null,"""Return to Owner""","""Neutered Male""",2555,7.0,"""(5.0, 7.5]""","""2014-12-20 16:35:00""",12,2014,"""2014-12""","""Saturday""",16,2.0,2007,7,"""2014-12""","""7 years""","""A006100""","""Dog""","""Spinone Italiano Mix""","""Yellow/White""","""8700 Research Blvd in Austin (…","""Normal""","""Public Assist""","""Neutered Male""",1,2555,7.0,"""(5.0, 7.5]""","""2014-12-19 10:21:00""",12,2014,"""2014-12""","""Friday""",10,2.0,"""1 days 06:14:00.000000000""",1.259722
"""6 years""","""A006100""","""2007-07-09 00:00:00""",null,"""Return to Owner""","""Neutered Male""",2190,6.0,"""(5.0, 7.5]""","""2014-03-08 17:10:00""",3,2014,"""2014-03""","""Saturday""",17,3.0,2007,7,"""2014-03""","""6 years""","""A006100""","""Dog""","""Spinone Italiano Mix""","""Yellow/White""","""8700 Research in Austin (TX)""","""Normal""","""Public Assist""","""Neutered Male""",1,2190,6.0,"""(5.0, 7.5]""","""2014-03-07 14:26:00""",3,2014,"""2014-03""","""Friday""",14,3.0,"""1 days 02:44:00.000000000""",1.113889
"""10 years""","""A047759""","""2004-04-02 00:00:00""","""Partner""","""Transfer""","""Neutered Male""",3650,10.0,"""(7.5, 10.0]""","""2014-04-07 15:12:00""",4,2014,"""2014-04""","""Monday""",15,1.0,2004,4,"""2014-04""","""10 years""","""A047759""","""Dog""","""Dachshund""","""Tricolor""","""Austin (TX)""","""Normal""","""Owner Surrender""","""Neutered Male""",1,3650,10.0,"""(7.5, 10.0]""","""2014-04-02 15:55:00""",4,2014,"""2014-04""","""Wednesday""",15,1.0,"""4 days 23:17:00.000000000""",4.970139
"""16 years""","""A134067""","""1997-10-16 00:00:00""",null,"""Return to Owner""","""Neutered Male""",5840,16.0,"""(15.0, 17.5]""","""2013-11-16 11:54:00""",11,2013,"""2013-11""","""Saturday""",11,1.0,1997,10,"""2013-11""","""16 years""","""A134067""","""Dog""","""Shetland Sheepdog""","""Brown/White""","""12034 Research Blvd in Austin …","""Injured""","""Public Assist""","""Neutered Male""",1,5840,16.0,"""(15.0, 17.5]""","""2013-11-16 09:02:00""",11,2013,"""2013-11""","""Saturday""",9,1.0,"""0 days 02:52:00.000000000""",0.119444


In [39]:
df.null_count()

age_upon_outcome,animal_id_outcome,date_of_birth,outcome_subtype,outcome_type,sex_upon_outcome,age_upon_outcome_(days),age_upon_outcome_(years),age_upon_outcome_age_group,outcome_datetime,outcome_month,outcome_year,outcome_monthyear,outcome_weekday,outcome_hour,outcome_number,dob_year,dob_month,dob_monthyear,age_upon_intake,animal_id_intake,animal_type,breed,color,found_location,intake_condition,intake_type,sex_upon_intake,count,age_upon_intake_(days),age_upon_intake_(years),age_upon_intake_age_group,intake_datetime,intake_month,intake_year,intake_monthyear,intake_weekday,intake_hour,intake_number,time_in_shelter,time_in_shelter_days
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,43324,10,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0


In [44]:
# drop rows with null outcomes and sex
df = df.drop_nulls(subset=["outcome_type", "sex_upon_intake"])

In [46]:
# dropping columns with redundant information, keeping days for all time columns
clean_df = df.drop(
    [
        "age_upon_outcome",
        "age_upon_outcome_(years)",
        "age_upon_outcome_age_group",
        "outcome_month",
        "outcome_monthyear",
        "outcome_weekday",
        "outcome_hour",
        "dob_year",
        "dob_monthyear",
        "age_upon_intake",
        "age_upon_intake_(years)",
        "age_upon_intake_age_group",
        "intake_month",
        "intake_monthyear",
        "intake_weekday",
        "intake_hour",
        "time_in_shelter",
    ]
)

# convert date columns to datetime type
clean_df = clean_df.with_columns(
    pl.col("date_of_birth").str.to_datetime(),
    pl.col("outcome_datetime").str.to_datetime(),
    pl.col("intake_datetime").str.to_datetime(),
)

clean_df.head()

animal_id_outcome,date_of_birth,outcome_subtype,outcome_type,sex_upon_outcome,age_upon_outcome_(days),outcome_datetime,outcome_year,outcome_number,dob_month,animal_id_intake,animal_type,breed,color,found_location,intake_condition,intake_type,sex_upon_intake,count,age_upon_intake_(days),intake_datetime,intake_year,intake_number,time_in_shelter_days
str,datetime[μs],str,str,str,i64,datetime[μs],i64,f64,i64,str,str,str,str,str,str,str,str,i64,i64,datetime[μs],i64,f64,f64
"""A006100""",2007-07-09 00:00:00,null,"""Return to Owner""","""Neutered Male""",3650,2017-12-07 14:07:00,2017,1.0,7,"""A006100""","""Dog""","""Spinone Italiano Mix""","""Yellow/White""","""Colony Creek And Hunters Trace…","""Normal""","""Stray""","""Neutered Male""",1,3650,2017-12-07 00:00:00,2017,1.0,0.588194
"""A006100""",2007-07-09 00:00:00,null,"""Return to Owner""","""Neutered Male""",2555,2014-12-20 16:35:00,2014,2.0,7,"""A006100""","""Dog""","""Spinone Italiano Mix""","""Yellow/White""","""8700 Research Blvd in Austin (…","""Normal""","""Public Assist""","""Neutered Male""",1,2555,2014-12-19 10:21:00,2014,2.0,1.259722
"""A006100""",2007-07-09 00:00:00,null,"""Return to Owner""","""Neutered Male""",2190,2014-03-08 17:10:00,2014,3.0,7,"""A006100""","""Dog""","""Spinone Italiano Mix""","""Yellow/White""","""8700 Research in Austin (TX)""","""Normal""","""Public Assist""","""Neutered Male""",1,2190,2014-03-07 14:26:00,2014,3.0,1.113889
"""A047759""",2004-04-02 00:00:00,"""Partner""","""Transfer""","""Neutered Male""",3650,2014-04-07 15:12:00,2014,1.0,4,"""A047759""","""Dog""","""Dachshund""","""Tricolor""","""Austin (TX)""","""Normal""","""Owner Surrender""","""Neutered Male""",1,3650,2014-04-02 15:55:00,2014,1.0,4.970139
"""A134067""",1997-10-16 00:00:00,null,"""Return to Owner""","""Neutered Male""",5840,2013-11-16 11:54:00,2013,1.0,10,"""A134067""","""Dog""","""Shetland Sheepdog""","""Brown/White""","""12034 Research Blvd in Austin …","""Injured""","""Public Assist""","""Neutered Male""",1,5840,2013-11-16 09:02:00,2013,1.0,0.119444
